In [1]:
%pip install openai langchain python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Cell 2: Load Environment Variables
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

# Cell 3: Define Prompt Generation Logic
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate

# Initialize OpenAI with API key
llm = OpenAI(api_key=openai_api_key)

def generate_prompts_and_responses(user_input, scenarios):
    template = """
    User input: {user_input}
    Scenario: {scenario}
    Generate a prompt for the above scenario.
    """
    prompts_and_responses = []
    for scenario in scenarios:
        # Generate the prompt
        prompt_template = PromptTemplate(template=template, input_variables=["user_input", "scenario"])
        chain = LLMChain(llm=llm, prompt=prompt_template)
        generated_prompt = chain.run(user_input=user_input, scenario=scenario)
        
        # Use the generated prompt to get a response
        response_chain = LLMChain(llm=llm, prompt=PromptTemplate(template="{prompt}", input_variables=["prompt"]))
        response = response_chain.run(prompt=generated_prompt)
        
        prompts_and_responses.append((generated_prompt, response))
    return prompts_and_responses

# Cell 4: Example Usage
user_input = "Generate a marketing slogan for an eco-friendly product"
scenarios = ["short and catchy", "emphasizes sustainability", "targeted at young adults"]
generated_prompts_and_responses = generate_prompts_and_responses(user_input, scenarios)

for prompt, response in generated_prompts_and_responses:
    print("Prompt:", prompt)
    print("Response:", response)
    print()


Prompt: 
"Create a memorable slogan for an eco-friendly product in just a few words!"
Response: 

"Go green, save the planet!" 


Prompt:  
"Create a memorable marketing slogan for a product that prioritizes sustainability and eco-friendliness." 
Response: 

"Go Green and Feel Good: Choose Our Sustainable Product!"

Prompt: 
"Create a catchy slogan for an eco-friendly product that appeals to young adults!"
Response: 

"Join the green team, keep the planet clean!"



In [11]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 919.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 812.0 kB/s eta 0:00:0000:0100:02
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
from dotenv import load_dotenv
import openai
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.base import Chain
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def generate_prompts(user_input, scenarios):
    template = "Given the user input: '{user_input}' and scenario: '{scenario}', generate a relevant prompt."
    prompts = []
    for scenario in scenarios:
        prompt_template = PromptTemplate(template=template, input_variables=["user_input", "scenario"])
        chain = LLMChain(llm=llm, prompt=prompt_template)
        result = chain.run(user_input=user_input, scenario=scenario)
        prompts.append(result)
    return prompts

def evaluate_prompts(user_input, prompts):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([user_input] + prompts)
    user_input_vector = vectors[0]
    prompt_vectors = vectors[1:]

    similarities = cosine_similarity(user_input_vector, prompt_vectors)
    evaluation_scores = similarities.flatten()
    return evaluation_scores

user_input = "Generate a marketing slogan for an eco-friendly product"
scenarios = ["short and catchy", "emphasizes sustainability", "targeted at young adults"]

# Generate prompts
generated_prompts = generate_prompts(user_input, scenarios)

# Evaluate prompts
evaluation_scores = evaluate_prompts(user_input, generated_prompts)

# Display prompts and their evaluation scores
for prompt, score in zip(generated_prompts, evaluation_scores):
    print(f"Prompt: {prompt}")
    print(f"Score: {score:.2f}\n")

Prompt: 

"Create a punchy tagline for an environmentally conscious product that will stick in people's minds."
Score: 0.13

Prompt: 

"Create a memorable tagline for an environmentally friendly product that highlights its commitment to sustainability."
Score: 0.20

Prompt: 

"Create a catchy marketing slogan for an eco-friendly product that will appeal to young adults and make a positive impact on the environment."
Score: 0.38



In [13]:
import numpy as np
import random

# Function to simulate matchups between prompts
def monte_carlo_matchmaking(prompts, evaluation_scores, num_simulations=100):
    num_prompts = len(prompts)
    wins = np.zeros(num_prompts)

    for _ in range(num_simulations):
        for i in range(num_prompts):
            for j in range(i + 1, num_prompts):
                if random.random() < evaluation_scores[i] / (evaluation_scores[i] + evaluation_scores[j]):
                    wins[i] += 1
                else:
                    wins[j] += 1

    return wins

# Function to calculate the expected score
def expected_score(rating1, rating2):
    return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

# Function to update ELO ratings
def update_elo_ratings(ratings, wins, num_simulations=100):
    K = 32
    for i in range(len(ratings)):
        for j in range(len(ratings)):
            if i != j:
                expected_i = expected_score(ratings[i], ratings[j])
                actual_i = wins[i] / num_simulations
                ratings[i] += K * (actual_i - expected_i)
    return ratings

# Main execution
user_input = "Generate a marketing slogan for an eco-friendly product"
scenarios = ["short and catchy", "emphasizes sustainability", "targeted at young adults"]

# Generate prompts
generated_prompts = generate_prompts(user_input, scenarios)

# Evaluate prompts
evaluation_scores = evaluate_prompts(user_input, generated_prompts)

# Monte Carlo matchmaking
num_simulations = 100
wins = monte_carlo_matchmaking(generated_prompts, evaluation_scores, num_simulations)

# Initial ELO ratings
initial_ratings = np.full(len(generated_prompts), 1500)

# Update ELO ratings based on matchmaking results
final_ratings = update_elo_ratings(initial_ratings, wins, num_simulations)

# Display prompts, their evaluation scores, and ELO ratings
for prompt, score, rating in zip(generated_prompts, evaluation_scores, final_ratings):
    print(f"Prompt: {prompt}")
    print(f"Score: {score:.2f}")
    print(f"ELO Rating: {rating:.2f}\n")

Prompt: 

"Get ready to go green with our eco-friendly product - the perfect solution for a sustainable lifestyle!"
Score: 0.24
ELO Rating: 1549.00

Prompt: 

"Create a catchy marketing slogan that promotes the eco-friendliness and sustainability of our product!"
Score: 0.27
ELO Rating: 1560.00

Prompt: 

"Create a catchy tagline promoting a sustainable product that will make young adults go green!"
Score: 0.04
ELO Rating: 1489.00

